In [ ]:
#@title Colab Extra Install { display-mode: "form" }
import os
import sys, re, requests
import platform

# Check if running in Colab
is_colab = any('COLAB_' in key for key in os.environ)
is_macos = platform.system() == "Darwin"

if is_colab:
    # Install uv in Colab
    !curl -LsSf https://astral.sh/uv/install.sh | sh
    # Add uv to PATH for Colab
    os.environ["PATH"] += os.pathsep + '/root/.cargo/bin'
    
    # Install GPU packages with uv in Colab
    !uv pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[uv pip install --no-deps unsloth vllm]]
    # Skip restarting message in Colab
    modules = list(sys.modules.keys())
    for x in modules:
        if "PIL" in x or "google" in x:
            sys.modules.pop(x, None)
    !uv pip install --no-deps bitsandbytes accelerate "xformers>=0.0.31" peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !uv pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    
    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !uv pip install -r vllm_requirements.txt
else:
    # Local development: use uv sync to install from pyproject.toml
    !uv sync --extra gpu
    print("Dependencies installed from pyproject.toml using uv sync")
    print("Platform-specific packages (GPU dependencies) are automatically handled")


Resolved 162 packages in 0.37ms
Audited 105 packages in 0.01ms
Dependencies installed from pyproject.toml using uv sync
Skipping GPU dependencies on macOS (use GPU-enabled environment for training)
To install GPU dependencies on Linux/Windows: uv sync --extra gpu


In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

ModuleNotFoundError: No module named 'unsloth'